<a href="https://csdms.colorado.edu"><img style="float: center; width: 75%" src="https://raw.githubusercontent.com/csdms/project/main/assets/CSDMS-logo-color-tagline-hor.png"></a>

# Object-Oriented Programming

*Object-oriented programming* (OOP) is a technique for modeling programming problems.
OOP is based on *objects*,
which bundle together data and functions that act upon these data,
and *classes*,
which describe what data and functions go into an object.
Compare this with *procedural programming*,
where data structures are separate from the functions that act upon them.
To this point,
we have mostly been using Python as a procedural language,
but Python also supports OOP.

In this lesson,
we'll go over some of the basics of OOP using Python.
This will be useful for EPSP scientists because
numerical models written with OOP techniques tend to be easier understand, debug, maintain, and extend.
Landlab is an example.
Further, if you intend to develop a Landlab component,
it must be written with OOP techniques.

## Concepts

<figure style="float: right;">
    <a href="https://en.wikipedia.org/wiki/File:Left_side_of_Flying_Pigeon.jpg"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Left_side_of_Flying_Pigeon.jpg/320px-Left_side_of_Flying_Pigeon.jpg" alt="A Flying Pigeon bicycle."/></a>
    <figcaption><i>Figure 1: A Flying Pigeon bicycle.</i></figcaption>
</figure>

Let's explore some OOP concepts through bicycles.

Bikes share many features.
They have wheels, handlebars, pedals, and brakes.
On a bicycle,
you can move, steer, and stop.

As a thought experiment,
consider a *class* called `Bike`.
Classes are used to organize information.
They are made up of data, called *attributes*,
and functions that act on these data, called *methods*.
In our `Bike` class,
wheels, handlebars, pedals, and brakes are attributes (what makes a `Bike`),
and the behaviors of move, steer, and stop are methods (what a `Bike` does).

Classes are generalizations.
An *object* is a particular instance of a class.
For example,
The Flying Pigeon in Figure 1 in an instance of a bike.
We can see that it has wheels, handlebars, pedals, and brakes,
and, in our thought experiment,
these attributes could be used in the behaviors of move, steer, and stop.

Let's go further.
Consider different kinds of bikes, such as mountain bikes versus road bikes.
Both mountain bikes and road bikes have all the attributes and methods of bikes,
but there are differences; for example,
mountain bikes tend to have wider tires and flat handlebars,
while road bikes have thinner tires and drop bars.
In OOP, we can handle this speciation through *inheritance*:
we can *subclass* `Bike` into new `MountainBike` and `RoadBike` classes.
These new classes have all the aspects of a `Bike`,
and they can define new ones,
such as a suspension attribute for a `MountainBike`.
An instance of a `MountainBike` is Mark's Specialized Stumpjumper FS.
An instance of a `RoadBike` is Benjamin's Bianchi Sprint.

Let's move from a conceptual example to a programmatic example.

## An example

For the rest of this lesson,
we'll use a textbook example--polygons--to see how OOP works in Python.

Figure 2 shows [class diagrams](https://en.wikipedia.org/wiki/Class_diagram) for four classes,
`Shape`, `Circle`, `Rectangle`, and `Square`.
`Shape` represents a generic polygon.
It is *abstract*. (More on that below.)
`Circle` and `Rectangle` are subclassed from `Shape`.
A square is a special case of a rectangle,
so the `Square` class is subclassed from `Rectangle`.

Refer back to this diagram as we explore the details of these classes below.

<figure>
    <img src="https://raw.githubusercontent.com/csdms/ivy/main/lessons/python/media/shapes-uml-diagram.png" alt="UML class diagrams for Shape, Circle, Rectangle, and Square."/>
    <figcaption><i>Figure 2: Class diagrams for Shape, Circle, Rectangle, and Square.</i></figcaption>
</figure>

Before we start, import NumPy, which we use several times in the example.

In [ ]:
import numpy as np

### Shape

Here's our definition of a shape.

In [ ]:
from abc import ABC, abstractmethod


class Shape(ABC):
    @abstractmethod
    def __init__(self, x: np.ndarray, y: np.ndarray) -> None:
        self.x = x
        self.y = y
        self.n_sides = len(x)
        self.area = None

    @abstractmethod
    def calculate_area(self) -> None: ...

Let's examine this code.

We start by importing a class and a decorator.
`ABC` is an acronym for *abstract base class*.
`Shape` inherits from `ABC`;
therefore, `Shape` is abstract,
meaning we will never create an instance of it.
Instead, we subclass `Shape` and create instances of the subclasses.

What happens when we try to create an instance of `Shape`?

In [ ]:
# Uncomment the next line to try to create a Shape.
# x = Shape()

`Shape` has two methods,
`__init__`, the class *constructor*,
and `calculate_area`,
a worker.
Both have the `@abstractmethod` [decorator](https://docs.python.org/3/glossary.html#term-decorator),
so they're also abstract and can't be called directly.
Instead, they're *overridden* in subclasses.
The constructor is called when we create an instance of a class--we never call `__init__` directly,
whether or not it's abstract.
In the signatures of the two methods,
we're using [type hints](https://docs.python.org/3/library/typing.html).
This is a choice;
they're useful, but not necessary.

Within a class definition,
the variable *self* is used to define and access attributes and methods.
In the constructor,
we set up four attributes of a shape: `x`, `y`, `n_sides`, and `area`.
Since we don't know how to calculate the area of a generic shape,
we leave the `calculate_area` method empty.
Subclasses of `Shape` will implement `calculate_area` using a formula for the particular shape.

### Circle

A circle is a shape.
Here's one way to define a `Circle` class.

In [ ]:
class Circle(Shape):
    def __init__(self, center: tuple = (1.0, 1.0), radius: float = 1.0) -> None:
        self.c = center
        self.r = radius
        theta = np.linspace(0.0, 2 * np.pi, 100)
        x = self.c[0] + self.r * np.cos(theta)
        y = self.c[1] + self.r * np.sin(theta)
        super().__init__(x, y)

    def calculate_area(self) -> None:
        self.area = np.pi * self.r**2

`Circle` is subclassed from `Shape`,
so it automatically inherits all of the `Shape`
attributes (`x`, `y`, `n_sides`, and `area`)
and
methods (`__init__` and `calculate_area`).

We can choose how to create a `Circle`.
Here, we use a center and a radius.
We then use the parametric equation for a circle to translate these
into the *(x,y)* coordinate pairs expected by `Shape`.
Since `Shape` is the *superclass* of `Circle`,
the inherited `Shape` constructor is called with the Python builtin function *super*.
The inherited `calculate_area` method is overridden with a formula for calculating the area of a circle.
Both the constructor and the `calculate_area` method are now *concrete*.
<!--
We actually invoke `Shape` ctr because it does stuff.
We don't bother to invoke the `calculate_area` method of the superclass.
It has to be overridden in the subclass, though, or Python will raise an exception.
-->

Make an instance of `Circle`, specifying a center and radius.

In [ ]:
circle = Circle(center=(1.0, 1.0), radius=2.0)

Show the values of the circle's center and radius attributes.

In [ ]:
circle.c

In [ ]:
circle.r

What's the area of the circle?

In [ ]:
circle.calculate_area()
circle.area

### Rectangle

There are many ways to define a rectangle.
We'll specify the lower left corner,
the width, and the height to make a `Rectangle` class.
Like `Circle`,
`Rectangle` inherits from `Shape`.

In [ ]:
class Rectangle(Shape):
    def __init__(
        self, lower_left: tuple = (1.0, 1.0), width: float = 3.0, height: float = 2.0
    ) -> None:
        self.ll = lower_left
        self.w = width
        self.h = height
        x = [self.ll[0], self.ll[0] + self.w, self.ll[0] + self.w, self.ll[0]]
        y = [self.ll[1], self.ll[1], self.ll[1] + self.h, self.ll[1] + self.h]
        super().__init__(x, y)

    def calculate_area(self) -> None:
        self.area = self.w * self.h

Note that most of the work in the constructor goes into forming arguments appropriate for `Shape`,
although creating *w* and *h* attributes for `Rectangle` helps with calculating the area.

Make a rectangle using the default argument values.

In [ ]:
rectangle = Rectangle()

Show the coordinates of the rectangle's vertices.

In [ ]:
rectangle.x

In [ ]:
rectangle.y

How many sides does the rectangle have?

In [ ]:
rectangle.n_sides

What's the area of the rectangle?

In [ ]:
rectangle.calculate_area()
rectangle.area

### Square

A square is a special case of a rectangle,
so it makes sense to subclass `Square` from `Rectangle`.
Inheritance makes the code really straightforward.

In [ ]:
class Square(Rectangle):
    def __init__(self, lower_left: tuple = (1.0, 1.0), width: float = 2.0) -> None:
        super().__init__(lower_left, width, width)

Note that the `Square` constructor simply calls the constructor of its superclass `Rectangle` with the *width* argument repeated. `Square` also doesn't need to define its own *calculate_area* method as it can use `Rectangle`'s.

Make a square:

In [ ]:
square = Square()

What are the coordinates of the square's vertices?

In [ ]:
for coordinate in zip(square.x, square.y):
    print(coordinate)

What's the area of the square?

In [ ]:
square.calculate_area()
square.area

## Exercises

Two suggestions:

1. Write a `Triangle` class. (How about a `Hexagon`?!)
1. Write code that uses Matplotlib to display an arbitrary `Shape`.

## Summary

It requires time to become adept at object-oriented programming,
but the time is usually well-spent
because code (think model code!) that is developed with OOP techniques
tends to be more robust and easier to test, maintain, and extend.

The concepts from this lesson are explored further in a recent CSDMS [webinar](https://youtu.be/dLrahDArm4w)
by Benjamin Campforts and Mark Piper.
You can find the slides used for the webinar [here](https://github.com/csdms/level-up/blob/master/info/Level-3-Object-oriented-Programming.pdf),
and a more elaborate example [here](https://github.com/BCampforts/python_corona_simulation)
(forked from [@paulvangentcom](https://github.com/paulvangentcom/python_corona_simulation)).

The table below summarizes the OOP terms used in this lesson.

| Term | Description |
| ---- | ----------- |
| object      | a variable that contains its own data + functions that act on the data |
| class       | the programming structure that defines what goes into an object |
| attribute   | data associated with an object |
| method      | a function that belongs to an object |
| instance    | synonym for object |
| constructor | the function called when an object is created |
| inheritance | the mechanism for deriving a new class from an existing class|
| subclass    | a class derived through inheritance |
| superclass  | the parent of a class |
| override    | when a subclass method takes precedence over the superclass method of the same name |
| abstract    | a class or method that can't be used directly; a prototype |
| concrete    | an implementation of an abstract class or method in a subclass |